In [740]:
import numpy as np
import random
import pandas as pd
from scipy.integrate import odeint
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.dates as mdates
from matplotlib import dates
from sklearn.metrics import mean_squared_error, r2_score
from datetime import datetime
from lmfit import Minimizer,minimize, Parameters, Parameter, report_fit
from statsmodels.tsa.arima_model import ARIMA

# step 0: data importation #

In [741]:
csv_url="https://raw.githubusercontent.com/ADelau/proj0016-epidemic-data/main/data.csv"
data = pd.read_csv(csv_url)

In [742]:
data.head()

,Day,num_positive,num_tested,num_hospitalised,num_cumulative_hospitalizations,num_critical,num_fatalities
0,1,0,0,1,1,0,0
1,2,5,8,1,1,0,0
2,3,10,12,2,2,0,0
3,4,11,16,2,2,0,0
4,5,9,12,2,2,0,0


# step 1: setting up the model #

## step 1.1: writing the ode system ##

In [743]:
def ode_model(z,t,params):
    N=sum(z)
    S,I,R = z
    beta_SI = params['beta_SI'].value
    gamma_IR = params['gamma_IR'].value
        
    #beta = beta_i*(1.1-tau*t) beta peut être une variable du temps
    dSdt = -beta_SI*(S*I)/(N)
    dIdt = +beta_SI*(S*I)/(N) - gamma_IR*I
    dRdt= +gamma_IR*I
    
    return [dSdt, dIdt, dRdt]

## step 1.2: writing the parameters, initial values and guesses ##


In [744]:
# the lmfit module uises an orderd dict structure
# to store the parameters to be optimized
# https://lmfit.github.io/lmfit-py/parameters.html

params = Parameters()
params.add('beta_SI', value=0.15, min=0.1, max=10)
params.add('gamma_IR', value= 0.05, min=0.1, max=10)

params.add('I0',value=15,min=1,max=500)

params.add('OBS_Tr_SI_to_nbTest',value=0.25,min=0.05,max=1)
params.add('OBS_nbTest_to_nbpos',value=0.75,min=0.5,max=0.9)





In [745]:
params

name,value,initial value,min,max,vary
beta_SI,0.15000000,0.15,0.10000000,10.0000000,True
gamma_IR,0.10000000,0.05,0.10000000,10.0000000,True
I0,15.0000000,15,1.00000000,500.000000,True
OBS_Tr_SI_to_nbTest,0.25000000,0.25,0.05000000,1.00000000,True
OBS_nbTest_to_nbpos,0.75000000,0.75,0.50000000,0.90000000,True


## step 1.3 writing the solver ##

In [746]:
#this function solves the ode
#input: function deriv, initial compartiments y0, t, and N,ps as arguments??
#output: the ode solution
def odesol(y,t,N,ps):
    
    ts = np.linspace(0, t, t+1)
    result = odeint(ode_model, y, ts, args=(N, ps))
    
    return result


def ode_solver(tspan, initial_conditions, params):
    #parvals = params.valuesdict()
        
    
    beta, gamma = params['beta_SI'].value, params['gamma_IR'].value
    
    
    res = odeint(ode_model, initial_conditions, tspan, args=(params,))
    return res

In [747]:
ts = np.linspace(0, 70,2)
ts

array([ 0., 70.])

In [748]:
test=ode_solver(ts, [1e6-50,50,0], params)
test

array([[9.99950000e+05, 5.00000000e+01, 0.00000000e+00],
       [9.95175171e+05, 1.63382576e+03, 3.19100320e+03]])

In [749]:
# this function generate the observation dataframe
def create_obsdf(model,ps,data):
    
    result=pd.DataFrame(columns=data.columns)
    result['Day']=data['Day']
    nb_trans_SI=ps['beta_SI']*(model['S']*(model['I']))/(model['S']+model['I']+model['R'])
    

    result['num_tested']=ps['OBS_Tr_SI_to_nbTest'].value*nb_trans_SI
    
    #result['num_hospitalised']=model['H']
    #result['num_cumulative_hospitalizations']=result['num_hospitalised'].diff().fillna(0).cumsum()
    #result['num_critical']=model['ICU']
    #result['num_fatalities']=model['D']
    return result
    

#test['num_tested']=params['OBS_Tr_EI_to_nbTest'].value*

# step 2: fitting the model #

## step 2.1: write objective function ##

In [750]:
#NOT USED, SEE RESIDUAL2
#this function compute residuals,
#ie objective function to be minimized in the optimization function
def residual(ps, ts, data, N):
    
    
    I0 = ps['I0'].value
    S0=N-I0
    R0=0

    
    y0 = S0,I0,R0
    
    model = pd.DataFrame(odesol(y0,ts,N,ps), columns=['S','I','R'])
    obs_df=create_obsdf(model,ps,data)
    component1=(obs_df['num_tested']-data['num_tested']).ravel()
    
    # penalty function to think more about !!!
    print(ps)
    return component1

In [759]:
#residual function from lmfit website
#ts is the x
def error(params, tspan, data):
    # unpack parameters: extract .value attribute for each parameter
    parvals = params.valuesdict()
    beta=parvals['beta_SI']
    gamma=parvals['gamma_IR']
    I0=parvals['I0']
    OBS_Tr_SI_to_nbTest=parvals['OBS_Tr_SI_to_nbTest']
    OBS_nbTest_to_nbpos=parvals['OBS_nbTest_to_nbpos']
    
    # useful scalars, arrays
    
    
    array_data=data['num_tested'].to_numpy()
    ## A CORRIGER !ode_solver(t, initial_conditions, params):
    array_model=ode_solver(tspan, [1e6-I0,I0,0],params)
    
    #array initialisation
    
    ratio_model=np.empty(array_data.shape[0]-1)
    ratio_data=np.empty(array_data.shape[0]-1)
    
    for i in range(array_model.shape[0]-1):
        ratio_model[i]=(array_model[i+1][1]-array_model[i][1])/array_model[i][1]
        ratio_data[i]=(array_data[i+1]-array_data[i])/array_model[i][1]
    
    return ratio_model-ratio_data

In [760]:
def find_timespane(data):
    return np.linspace(0,data.shape[0]-1,data.shape[0])
def find_timespane2(data):
    return np.array(data.index.tolist())

In [761]:
error_array=error(params,find_timespane(data),data)

In [762]:
error_array

array([-0.48206478, -0.20239362, -0.19002345,  0.28079142,  0.16043234,
        0.10318736, -0.59078944,  0.00428512,  0.1406437 , -0.62889691,
        0.21301223,  0.05126322, -0.13168507, -0.19237533, -0.21360388,
        0.20872906, -0.21836294, -0.09122771, -0.16560438,  0.30911751,
       -0.19403173,  0.19125201, -0.32606465,  0.26238368, -0.39059049,
       -0.02516794,  0.08759555,  0.03396051, -0.14608776, -0.32418401,
       -0.05291907,  0.33434117, -0.62201085,  0.25617711,  0.01468282,
       -0.40079714,  0.01815486, -0.45220968,  0.10111315, -0.00572126,
       -0.30088984, -0.39538712,  0.32900043, -0.51614701, -0.1484121 ,
       -0.0964913 , -0.36361626,  0.11484701, -0.41501946, -0.13888387,
       -0.4364762 , -0.14689412, -0.09758444,  0.04645562, -0.51427949,
       -0.15376087, -0.35498744, -0.28499873, -0.1914567 , -0.09223667,
       -0.39133404, -0.41096346, -0.18374655, -0.11504142, -0.42311055,
       -0.31190991, -0.41514999, -0.22587008, -0.2235894 , -0.30

## step 2.2: write optimization command ##

In [763]:
test = minimize(error, params, args=(tspan, data), method ='leastsq')

In [765]:
test

In [758]:
tspan=find_timespane(data)

mini = Minimizer(error, params, args=(initial_condition, tspan, data), method ='leastsq')
result = mini.minimize()
#result

TypeError: leastsq() got an unexpected keyword argument 'method'